In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
!pip -q install pyngrok
!pip -q install streamlit
!pip -q install patool

import cv2
import gdown
import glob
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import patoolib

from joblib import dump
from pyngrok import ngrok
from tqdm import tqdm

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
!unzip archive.zip

In [ ]:
!unzip PoisonIvy.zip

In [ ]:
train_dir = 'RashData/Train/Train_2_Cases'
val_dir = 'RashData/Validation/Validation_2_Cases'

In [ ]:
trainpi = 'PoisonIvy/train'
testpi = 'PoisonIvy/test'

In [ ]:
batch_size = 32
img_height = 128
img_width = 128

In [ ]:
# Lyme Disease:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    seed=42,
    subset='training',
    batch_size=batch_size,
    image_size=(32, 32)
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    seed=42,
    subset='validation',
    batch_size=batch_size,
    image_size=(32, 32)
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    batch_size=batch_size,
    image_size=(32, 32)
)

In [ ]:
# Poison Ivy:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    trainpi,
    validation_split=0.2,
    seed=42,
    subset='training',
    batch_size=batch_size,
    image_size=(32, 32)
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    testpi,
    batch_size=batch_size,
    image_size=(32, 32)
)

In [ ]:
class_names = train_ds.class_names

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_datset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
inputs = tf.keras.Input(shape=(32, 32, 3))
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

In [ ]:
modelpi = tf.keras.Model(inputs, outputs)

modelpi.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics=[
             'accuracy',
             tf.keras.metrics.AUC(name = 'auc')
    ]
)

epochs = 6

historypi = modelpi.fit(
    train_dataset,
    validation_data=test_dataset,
    batch_size=batch_size,
    epochs=epochs,
)

In [ ]:
from keras.layers import Dense, Activation, Flatten, Dropout

inputs = tf.keras.Input(shape=(32, 32, 3))
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='sigmoid')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = Dropout(0.5)(x)

x = tf.keras.layers.Conv2D(16, 3, padding='same', activation='sigmoid')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = Dropout(0.5)(x)

x = tf.keras.layers.Conv2D(16, 3, padding='same', activation='sigmoid')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = Dropout(0.5)(x)

x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='sigmoid')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = Dropout(0.5)(x)

x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = Dropout(0.5)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

In [ ]:
model_ld = tf.keras.Model(inputs, outputs)

model_ld.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics=[
             'accuracy',
             tf.keras.metrics.AUC(name = 'auc')
    ]
)

epochs = 8

history_ld = model_ld.fit(
    train_ds,
    validation_data=val_ds,
    batch_size=batch_size,
    epochs=epochs,
)

In [ ]:
DATA_ROOT = '/content'
os.makedirs(DATA_ROOT, exist_ok=True)
max_samples = 3000

blood_slide_url = 'https://drive.google.com/uc?id=1lffxAG8gykh1dh1pCP34uRkH3XMwuNt-'
blood_slide_path = os.path.join(DATA_ROOT, 'blood_slide.jpg')
gdown.download(blood_slide_url, blood_slide_path, True)

!wget "https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/Deep%20Dives/malaria_images.zip"
!unzip "malaria_images.zip"
malaria_imgs_path = os.path.join(DATA_ROOT, 'malaria_images.zip')

u_malaria_img_paths = glob.glob('/content/malaria_images/Uninfected/*png')
p_malaria_img_paths = glob.glob('/content/malaria_images/Parasitized/*png')

NUM_SAMPLES = len(u_malaria_img_paths) + len(p_malaria_img_paths)

X = []
y = []

X_g = []

for i in tqdm(range(max_samples)):
  img = cv2.imread(u_malaria_img_paths[i])
  X.append(cv2.resize(img,(50,50)))

  gray_img = cv2.imread(u_malaria_img_paths[i],0)
  X_g.append(cv2.resize(gray_img,(50,50)))

  y.append(0)

for i in tqdm(range(max_samples)):
  img = cv2.imread(p_malaria_img_paths[i])
  X.append(cv2.resize(img,(50,50)))

  gray_img = cv2.imread(p_malaria_img_paths[i],0)
  X_g.append(cv2.resize(gray_img,(50,50)))

  y.append(1)

X = np.stack(X)
X_g = np.stack(X_g)
X_reshaped = np.reshape(X_g,(X_g.shape[0],2500))

y = np.array(y)

blood_samples_dir = 'blood_samples'
if (os.path.exists(blood_samples_dir) == False):
  os.mkdir(blood_samples_dir)

for i, img in enumerate(X[2995:3005]):
  plt.imsave('test_img_{}.jpg'.format(i), img)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size = 0.33, random_state = 42)
model = SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)
model = model.fit(X_train, y_train)
dump(model, "model.joblib")

In [ ]:
dump(modelpi, "model.joblib2")
dump(model_ld, "model.joblib3")

In [ ]:
!pip install streamlit-multipage

In [ ]:
%%writefile app.py
import streamlit as st
from joblib import load
import numpy as np
import cv2
import time
from PIL import Image
import base64
from streamlit_multipage import MultiPage

def info_page(st, **state):
  st.title("Additional Information - Slideshow")
  st.write("")
  st.write("")
  with open("Remote Disease Detector.pdf","rb") as f:
    base64_pdf = base64.b64encode(f.read()).decode('utf-8')
  pdf_display = F'<iframe src="data:application/pdf;base64,{base64_pdf}" width="700" height="1000" type="application/pdf"></iframe>'
  st.markdown(pdf_display, unsafe_allow_html=True)

def image_page(st, **state):
  st.title("Disease Image Classification")
  model = load("model.joblib")
  modelpi = load("model.joblib2")
  model_ld = load("model.joblib3")

  st.title('Lyme Disease Diagnosis')

  uploaded_file = st.file_uploader("Upload Lyme Disease Rash File")

  if uploaded_file is not None:
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, 1)
    small = cv2.resize(image, (32, 32))
    small_batch = np.expand_dims(small, 0)
    output = model_ld.predict(small_batch)
    if output > 0.5257:
      st.write("Negative: No Lyme Disease was detected.")
    else:
      st.write("Positive: Lyme Disease was detected.")
      st.write("Please contact your local health care provider and follow the CDC guidlines for more information: https://www.cdc.gov/malaria/travelers/drugs.html")

  st.title('Malaria Diagnosis')
  uploaded_file2 = st.file_uploader("Upload Malaria Blood Cell File")

  if uploaded_file2 is not None:
    file_bytes = np.asarray(bytearray(uploaded_file2.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, 1)

    small = cv2.resize(image, (50, 50))
    gray = cv2.cvtColor(small, cv2.COLOR_BGR2GRAY)

    gray = np.reshape(gray, (1, 2500))
    output = model.predict(gray)[0]
    if output == 0:
      st.write("Negative: No malaria was detected.")
    else:
      st.write("Positive: Malaria was detected.")
      st.write("Please contact your local health care provider and follow the CDC guidlines for more information: https://www.cdc.gov/malaria/travelers/drugs.html")

  st.title('Poison Ivy Diagnosis')
  uploaded_file3 = st.file_uploader("Upload Poison Ivy Rash File")

  if uploaded_file3 is not None:
    file_bytes = np.asarray(bytearray(uploaded_file3.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, 1)
    small = cv2.resize(image, (32, 32))
    small_batch = np.expand_dims(small, 0)

    output = modelpi.predict(small_batch)
    if output < 0.2:
      st.write("Negative: No Poison Ivy was detected.")
    else:
      st.write("Positive: Poison Ivy was detected.")
      st.write("Please contact your local health care provider and follow the CDC guidlines for more information: https://www.cdc.gov/malaria/travelers/drugs.html")

def home_page(st, **state):
  st.title("Disease Detector Home Page")
  st.write("Our mission is to provide those without immediate access to healthcare professionals quick diagnosis for a plethora of diseases.")
  st.write("")
  image = Image.open('homepagepic.jpeg')
  st.image(image, caption='**Through the use of machine learning algorithms such as Convolutional Neural Networks and Support Vector Machines, our project strives to diagnose diseases/illnesses for users.**')

app = MultiPage()
app.st = st
app.navbar_name = "Pages:"
app.navbar_style = "SelectBox"
app.reset_button = "-------------"

app.add_app("Disease Detector Home Page", home_page)
app.add_app("Image Classification", image_page)
app.add_app("Self-Diagnosis Test", akinator_page)
app.add_app("Additional Information", info_page)
app.run()

In [ ]:
!ngrok authtoken "26IEGw1HiDLy0wj5uqrav4wIEaA_2mh3GVNpmJD7VMa2bRMWZ"

In [ ]:
public_url = ngrok.connect(port='80')
print(public_url)
!streamlit run --server.port 80 app.py >/dev/null